# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

In [2]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.2*len(df))]
update_df = df[int(0.2*len(df)):int(0.7*len(df))]
dev_df = df[int(0.7*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Dev dataframe shape: {dev_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Train dataframe shape: (206, 10)
Update dataframe shape: (515, 10)
Dev dataframe shape: (103, 10)
Test dataframe shape: (206, 10)


In [3]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

jai.outputs[target].mixers = [{
    "module": "Neural",
    "args": {
        "fit_on_dev": False,
        "stop_after": "$problem_definition.seconds_per_mixer",
        "search_hyperparameters": False,
    }
}]


predictor = predictor_from_json_ai(jai)

# Train and get predictions for the held out test set
predictor.learn(train_df)
predictions = predictor.predict(test_df)
predictions

INFO:lightwood-67282:Dropping features: []
INFO:lightwood-67282:Analyzing a sample of 979
INFO:lightwood-67282:from a total population of 1030, this is equivalent to 95.0% of your data.
INFO:lightwood-67282:Using 15 processes to deduct types.
INFO:lightwood-67282:Starting statistical analysis
INFO:lightwood-67282:Finished statistical analysis
INFO:lightwood-67282:Unable to import black formatter, predictor code might be a bit ugly.
INFO:lightwood-67282:Dropping features: []
INFO:lightwood-67282:Performing statistical analysis on data
INFO:lightwood-67282:Starting statistical analysis
INFO:lightwood-67282:Finished statistical analysis
INFO:lightwood-67282:Cleaning the data
INFO:lightwood-67282:Splitting the data into train/test
INFO:lightwood-67282:Preparing the encoders
INFO:lightwood-67282:Encoder prepping dict length of: 1
INFO:lightwood-67282:Encoder prepping dict length of: 2
INFO:lightwood-67282:Encoder prepping dict length of: 3
INFO:lightwood-67282:Encoder prepping dict length o

DEBUG:lightwood-67282:Loss @ epoch 72: 0.05157444253563881
DEBUG:lightwood-67282:Loss @ epoch 73: 0.05137106031179428
DEBUG:lightwood-67282:Loss @ epoch 74: 0.05131785199046135
DEBUG:lightwood-67282:Loss @ epoch 75: 0.05133713781833649
DEBUG:lightwood-67282:Loss @ epoch 76: 0.05156172439455986
INFO:lightwood-67282:Ensembling the mixer
INFO:lightwood-67282:Mixer: Neural got accuracy: 0.5960601553597429
INFO:lightwood-67282:Picked best mixer: Neural
INFO:lightwood-67282:Analyzing the ensemble of mixers
INFO:lightwood-67282:The block ICP is now running its analyze() method
INFO:lightwood-67282:The block AccStats is now running its analyze() method
INFO:lightwood-67282:The block GlobalFeatureImportance is now running its analyze() method
INFO:lightwood-67282:Adjustment on validation requested.
INFO:lightwood-67282:Updating the mixers
torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
DEBUG:lightwood-67282:Loss @ epoch 1: 0.06892643496394157
DEBUG:lightwood-67282:L

,prediction,truth,confidence,lower,upper
0,51.193603,71.30,0.9991,30.540443,71.846764
1,28.503390,39.60,0.9991,7.850229,49.156551
2,18.356139,10.79,0.9991,0.000000,39.009300
3,16.062094,4.83,0.9991,0.000000,36.715254
4,32.623629,47.71,0.9991,11.970469,53.276790
...,...,...,...,...,...
201,45.633811,40.93,0.9991,24.980650,66.286972
202,41.613209,52.82,0.9991,20.960048,62.266369
203,31.297044,39.66,0.9991,10.643883,51.950204
204,29.409258,13.29,0.9991,8.756097,50.062418


## Updating the predictor

We've explicitly configured our predictor to use a `BestOf` ensemble that will select the only mixer we've passed, a neural network-based mixer. Let's confirm this:

In [4]:
predictor.ensemble

In [5]:
mixer = predictor.ensemble.mixers[predictor.ensemble.best_index]
mixer

We will be updating this neural network mixer with new data. 

As previously mentioned, all we need is a `partial_fit()` call. As input, it requires two encoded datasources. In this case, we will pass the `update_df` data as the new training data to finetune this predictor, and will use the `dev_df` data split for internal model usage:

In [6]:
from lightwood.data import EncodedDs

update_ds = EncodedDs(predictor.encoders, update_df, target)
dev_ds = EncodedDs(predictor.encoders, dev_df, target)

mixer.partial_fit(update_ds, dev_ds)

torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
DEBUG:lightwood-67282:Loss @ epoch 1: 0.06531730045874913
DEBUG:lightwood-67282:Loss @ epoch 2: 0.0703464150428772
DEBUG:lightwood-67282:Loss @ epoch 3: 0.0612030898531278
DEBUG:lightwood-67282:Loss @ epoch 4: 0.06489518160621326
DEBUG:lightwood-67282:Loss @ epoch 5: 0.06311412776509921
DEBUG:lightwood-67282:Loss @ epoch 6: 0.057933310667673744
DEBUG:lightwood-67282:Loss @ epoch 7: 0.06008055806159973
DEBUG:lightwood-67282:Loss @ epoch 8: 0.06116113935907682


In [7]:
new_predictions = predictor.predict(test_df)
new_predictions

INFO:lightwood-67282:Dropping features: []
INFO:lightwood-67282:Cleaning the data
INFO:lightwood-67282:Featurizing the data
INFO:lightwood-67282:The block ICP is now running its explain() method
INFO:lightwood-67282:The block AccStats is now running its explain() method
INFO:lightwood-67282:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-67282:The block GlobalFeatureImportance is now running its explain() method
INFO:lightwood-67282:GlobalFeatureImportance.explain() has not been implemented, no modifications will be done to the data insights.


,prediction,truth,confidence,lower,upper
0,53.567344,71.30,0.9991,32.914183,74.220505
1,28.273047,39.60,0.9991,7.619886,48.926207
2,16.170064,10.79,0.9991,0.000000,36.823225
3,13.872111,4.83,0.9991,0.000000,34.525272
4,30.346309,47.71,0.9991,9.693149,50.999470
...,...,...,...,...,...
201,44.517547,40.93,0.9991,23.864386,65.170707
202,36.844431,52.82,0.9991,16.191270,57.497591
203,28.707179,39.66,0.9991,8.054018,49.360339
204,28.481116,13.29,0.9991,7.827955,49.134276


Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies:

In [8]:
from sklearn.metrics import r2_score

old_acc = r2_score(predictions['truth'], predictions['prediction'])
new_acc = r2_score(new_predictions['truth'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

Old Accuracy: 0.583
New Accuracy: 0.604


After updating, we see an increase in the R2 score of predictions for the held out test set.

## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.